In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV,LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import Normalizer

In [2]:
from fancyimpute import KNN

Using TensorFlow backend.


In [3]:
data = pd.read_csv("dataCase1.csv")
data.head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100
0,-4.551892,-1.809625,-0.913360,-0.218107,0.687713,-1.357434,3.087521,0.134304,5.501414,0.480548,...,1.935061,-0.364959,0.228810,-0.145521,-1.754263,B,A,C,C,H
1,1.806876,NaN,-2.067355,-0.259221,1.682711,-0.899337,-0.448752,0.246285,2.007544,1.669644,...,2.129238,2.873889,-0.956736,2.946482,0.348536,B,B,D,B,B
2,-7.418004,3.145955,-1.434793,-2.355750,2.145708,-2.730022,0.181677,-1.603840,1.481230,0.863563,...,3.011292,0.023706,-0.118926,-1.329891,-0.549678,B,B,B,D,E
3,-8.776398,-0.966571,2.768401,-1.016546,2.579520,-2.989883,1.299483,0.537700,1.972970,0.214713,...,2.334895,0.131329,-1.506334,-1.198288,-0.061236,B,B,A,B,D
4,-6.755842,-0.998227,0.657886,-1.104707,1.289966,-3.907128,0.901370,0.497693,1.794773,0.779314,...,3.131590,-1.299633,-0.861064,0.672235,-0.659976,A,C,D,D,D


In [4]:
#convert dataframe to np array
dataset = data.values

In [5]:
#filling the numirical missing values with mean

# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# fill_x_numbers = imp.fit_transform(dataset[:,1:96])


#filling the numirical missing values with KNN

fill_x_numbers = KNN(k=8).fit_transform(dataset[:,1:96])




Imputing row 1/1100 with 1 missing, elapsed time: 0.578
Imputing row 101/1100 with 0 missing, elapsed time: 0.605
Imputing row 201/1100 with 1 missing, elapsed time: 0.607
Imputing row 301/1100 with 2 missing, elapsed time: 0.609
Imputing row 401/1100 with 1 missing, elapsed time: 0.611
Imputing row 501/1100 with 1 missing, elapsed time: 0.613
Imputing row 601/1100 with 4 missing, elapsed time: 0.614
Imputing row 701/1100 with 1 missing, elapsed time: 0.616
Imputing row 801/1100 with 1 missing, elapsed time: 0.618
Imputing row 901/1100 with 1 missing, elapsed time: 0.620
Imputing row 1001/1100 with 0 missing, elapsed time: 0.621


In [6]:
#filling the categorial with mode
imp2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
fill_x_categorial =  imp2.fit_transform(dataset[:,96:])

In [7]:
fill_x_numbers

array([[-1.80962522e+00, -9.13360030e-01, -2.18106911e-01, ...,
         2.28809829e-01, -1.45521342e-01, -1.75426281e+00],
       [ 3.51332834e-01, -2.06735518e+00, -2.59220596e-01, ...,
        -9.56735893e-01,  2.94648201e+00,  3.48536270e-01],
       [ 3.14595538e+00, -1.43479340e+00, -2.35575046e+00, ...,
        -1.18925719e-01, -1.32989134e+00, -5.49677796e-01],
       ...,
       [ 2.53341661e+00,  2.67357206e+00, -2.69006327e+00, ...,
         2.82892960e-01,  2.35787325e-01, -1.13154608e+00],
       [-2.15414378e-01, -2.14595750e+00, -5.54640373e-01, ...,
        -1.87423200e+00,  9.47279120e-01, -2.20062091e+00],
       [ 2.11332738e+00,  3.00334642e-01, -3.22017017e-01, ...,
        -6.13649100e-01, -4.73622369e-01, -2.05984552e-03]])

In [8]:
#concate the two
X_filled = np.concatenate((fill_x_numbers, fill_x_categorial), axis=1)

In [9]:
print(X_filled)

[[-1.80962521634919 -0.913360029753062 -0.218106910804778 ... 'C' 'C' 'H']
 [0.35133283440293606 -2.0673551820964198 -0.259220595873348 ... 'D' 'B'
  'B']
 [3.14595538020002 -1.43479340320342 -2.35575045541893 ... 'B' 'D' 'E']
 ...
 [2.53341661067358 2.6735720627366297 -2.6900632684599994 ... 'C' 'C' 'G']
 [-0.215414378225097 -2.14595750481143 -0.554640373174789 ... 'C' 'B' 'E']
 [2.11332738439507 0.30033464189362896 -0.32201701712570296 ... 'C' 'A'
  'F']]


In [10]:
#using one out of k coding for categorial values
enc = OneHotEncoder(handle_unknown='ignore')

In [11]:
encoding_categorials = enc.fit_transform(X_filled[:,95:]).toarray()

In [12]:
encoding_categorials

array([[0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.]])

In [13]:
X_filled = np.concatenate((X_filled[:,1:95], encoding_categorials), axis=1)

In [14]:
X_filled

array([[-0.913360029753062, -0.218106910804778, 0.6877134470637329, ...,
        0.0, 0.0, 1.0],
       [-2.0673551820964198, -0.259220595873348, 1.68271062536683, ...,
        0.0, 0.0, 0.0],
       [-1.43479340320342, -2.35575045541893, 2.14570810586986, ..., 0.0,
        0.0, 0.0],
       ...,
       [2.6735720627366297, -2.6900632684599994, 2.31023096931002, ...,
        0.0, 1.0, 0.0],
       [-2.14595750481143, -0.554640373174789, 0.7319332081062471, ...,
        0.0, 0.0, 0.0],
       [0.30033464189362896, -0.32201701712570296, 0.356156389277952,
        ..., 1.0, 0.0, 0.0]], dtype=object)

In [15]:
#observation with label
Y = dataset[0:100,0]
X = X_filled[0:100,:]

#observation without label
X_new = X_filled[100:,:]

In [16]:
len(Y)

100

In [17]:
#feature selection

estimator = SVR(kernel="linear")
selector = RFE(estimator)
selector = selector.fit(X,Y)

In [18]:
X_feature = selector.transform(X)
print(X_feature)

[[-0.218106910804778 0.6877134470637329 -1.3574335108351299 ... 0.0 0.0
  1.0]
 [-0.259220595873348 1.68271062536683 -0.899336590157309 ... 0.0 0.0 0.0]
 [-2.35575045541893 2.14570810586986 -2.73002172725291 ... 0.0 1.0 0.0]
 ...
 [-0.21867481287381102 2.5030396698436 -1.45336124316846 ... 0.0 0.0 0.0]
 [1.34547648462266 0.9659790495862209 -0.878476431384007 ... 0.0 1.0 0.0]
 [0.334669295547417 2.5672019826376604 -0.195528932324772 ... 0.0 0.0 0.0]]


In [19]:
X_feature.shape

(100, 57)

In [20]:
#using cross validation for different models
X_train, X_test, y_train, y_test = train_test_split(
    X_feature, Y, test_size=0.33, random_state=42)

In [21]:
#ridge regression with cross validation
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1, 10]).fit(X_train, y_train)
clf.score(X_test, y_test)

0.8308669828764477

In [22]:
#lasso regression with cross validation
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
reg.score(X_test, y_test)

c:\users\tosodoulis\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\tosodoulis\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.7068682966959563

In [23]:
#Knn regression

neigh = KNeighborsRegressor(n_neighbors=int(10))
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

0.3270959876553048

In [24]:
cols = selector.get_support(indices=True)

In [25]:
print(cols,len(cols))

[  1   2   3   4   5   9  11  12  13  14  20  22  25  26  27  30  31  32
  35  37  39  40  41  42  43  44  47  48  55  61  64  66  68  71  72  73
  74  76  80  82  83  85  88  91  93  94  95  96  98 103 105 106 107 108
 110 111 114] 57


In [26]:
X_new = X_new[:,cols]

In [27]:
X_new.shape

(1000, 57)

In [28]:
y_new = clf.predict(X_new)

In [29]:
print(len(y_new))

1000
